## Estadística Aplicada

Actividad 6A - Grupo 41

David Eduardo Gómez Marín - 1914352


# 1. Importa las librerías básicas y especializadas que consideres necesarias


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from lifelines import CoxPHFitter

# 2. Importa la base de datos https://raw.githubusercontent.com/jimmyzac/EstadisticaAplicada-FCFM-UANL/main/bases_datos/desercion_laboral.csv

In [2]:
renuncias = pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/desercion_laboral.csv')

In [3]:
renuncias

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario
0,3.8,5.3,2,157,3,0,1,0,ventas,bajo
1,8.0,8.6,5,262,6,0,1,0,ventas,medio
2,1.1,8.8,7,272,4,0,1,0,ventas,medio
3,7.2,8.7,5,223,5,0,1,0,ventas,bajo
4,3.7,5.2,2,159,3,0,1,0,ventas,bajo
...,...,...,...,...,...,...,...,...,...,...
14994,4.0,5.7,2,151,3,0,1,0,soporte,bajo
14995,3.7,4.8,2,160,3,0,1,0,soporte,bajo
14996,3.7,5.3,2,143,3,0,1,0,soporte,bajo
14997,1.1,9.6,6,280,4,0,1,0,soporte,bajo


# 3. Depurar la base de datos y estadísticas descriptivas
a) Verificar que no haya missing values, en caso de haberlos decidir imputar 
o eliminar

In [4]:
renuncias.isnull().sum()

satisfaccion           0
evaluacion             0
proyectos              0
promedio_horas_mes     0
semestre               0
accidente              0
renunciar              0
ascenso_5years         0
departamento          26
salario                0
dtype: int64

Hay 26 missing values de un total de 14,999 observaciones, por lo que eliminar estos missing values no representa una pérdida de información que realmente afecte al modelo. Por lo que decido eliminarlos a continuación.

In [5]:
renuncias = renuncias.dropna()
renuncias.isnull().sum()

satisfaccion          0
evaluacion            0
proyectos             0
promedio_horas_mes    0
semestre              0
accidente             0
renunciar             0
ascenso_5years        0
departamento          0
salario               0
dtype: int64

b) Verificar que no haya valores duplicados con dataframe.duplicated().sum()
en caso de haber valores duplicados eliminarlos con 
dataframe=dataframe.drop_duplicates(keep='first').reset_index(drop=True)


In [6]:
renuncias.duplicated().sum()

2982

In [7]:
renuncias = renuncias.drop_duplicates(keep='first').reset_index(drop=True)
renuncias.duplicated().sum()

0

c) Verificar que todas las variables sean numéricas, si no son numéricas 
hacerlas numéricas, en el caso de las categóricas convertirlas en 
dummies y conservar n-1 dummies


In [8]:
renuncias.dtypes

satisfaccion          float64
evaluacion            float64
proyectos               int64
promedio_horas_mes      int64
semestre                int64
accidente               int64
renunciar               int64
ascenso_5years          int64
departamento           object
salario                object
dtype: object

In [9]:
dummiesDepas = pd.get_dummies(renuncias['departamento'], prefix='depa').astype(int)
dummiesSalary = pd.get_dummies(renuncias['salario'], prefix='Salario').astype(int)

In [10]:
dummiesSalary

,Salario_alto,Salario_bajo,Salario_medio
0,0,1,0
1,0,0,1
2,0,0,1
3,0,1,0
4,0,1,0
...,...,...,...
11986,0,1,0
11987,0,1,0
11988,0,1,0
11989,0,1,0


In [11]:
dummiesDepas

,depa_Investigación,depa_administración,depa_contabilidad,depa_desarrollo de producto,depa_informática,depa_mantenimiento,depa_mercadotecnia,depa_recursos humanos,depa_soporte,depa_ventas
0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
11986,0,0,0,1,0,0,0,0,0,0
11987,0,0,0,1,0,0,0,0,0,0
11988,0,0,0,1,0,0,0,0,0,0
11989,0,0,0,1,0,0,0,0,0,0


In [12]:
dummiesDepas = dummiesDepas[['depa_Investigación', 'depa_administración', 'depa_contabilidad', 'depa_desarrollo de producto', 'depa_informática', 'depa_mantenimiento', 'depa_mercadotecnia', 'depa_recursos humanos', 'depa_soporte']]
dummiesSalary = dummiesSalary[['Salario_alto', 'Salario_medio']]

In [13]:
renuncias = pd.concat([renuncias, dummiesDepas, dummiesSalary], axis=1)

In [14]:
renuncias = renuncias.drop('departamento', axis=1)
renuncias = renuncias.drop('salario', axis=1)

In [15]:
renuncias.head()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,depa_Investigación,depa_administración,depa_contabilidad,depa_desarrollo de producto,depa_informática,depa_mantenimiento,depa_mercadotecnia,depa_recursos humanos,depa_soporte,Salario_alto,Salario_medio
0,3.8,5.3,2,157,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,8.0,8.6,5,262,6,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,1.1,8.8,7,272,4,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,7.2,8.7,5,223,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,3.7,5.2,2,159,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0


d) Calcular estadísticas descriptivas


In [16]:
renuncias.describe()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,depa_Investigación,depa_administración,depa_contabilidad,depa_desarrollo de producto,depa_informática,depa_mantenimiento,depa_mercadotecnia,depa_recursos humanos,depa_soporte,Salario_alto,Salario_medio
count,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.00000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000
mean,6.296581,7.166825,3.802852,200.473522,3.364857,0.154282,0.166041,0.016929,0.057877,0.036361,0.051789,0.057210,0.081394,0.18714,0.056125,0.050121,0.151864,0.082562,0.438746
std,2.410700,1.683426,1.163238,48.727813,1.330240,0.361234,0.372133,0.129012,0.233520,0.187194,0.221610,0.232252,0.273451,0.39004,0.230173,0.218204,0.358904,0.275230,0.496254
min,0.900000,3.600000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.800000,5.700000,3.000000,157.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.600000,7.200000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.200000,8.600000,5.000000,243.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
max,10.000000,10.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


En promedio, los empleados han trabajado 3.36 semestres en la empresa. El empleado que más tiempo ha trabajado lleva 10 semestres en la empresa mientras que los mas nuevos llevan 2 semestres.

El 15.43% de los empleados ha sufrido accidentes en la empresa.

El 16.6% de los empleados ha renunciado.

Solo el 1.69% de los empleados ha tenido un ascenso en los últimos 5 años

Solo el 8.25% de los empleados cuenta con un salario alto, mientras que el 43.87% cuenta con un salario medio, lo que significa que casi el 50% de los empleados sigue con un salario bajo, esto, aunado a que solo el 1.69% ha recibido un ascenso en 5 años, es un muy mal dato.


# 4. Estimar el modelo de Cox e interpretar cada uno de los coeficientes


In [17]:
cph1 = CoxPHFitter().fit(renuncias, 'semestre', 'renunciar')
cph1.print_summary()

<lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
      baseline estimation = breslow
   number of observations = 11991
number of events observed = 1991
   partial log-likelihood = -15884.05
         time fit was run = 2023-09-25 08:33:08 UTC

---
                              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                           
satisfaccion                 -0.24       0.78       0.01            -0.26            -0.22                 0.77                 0.80
evaluacion                   -0.01       0.99       0.01            -0.04             0.02                 0.96                 1.02
proyectos                    -0.29       0.75       0.02            -0.33            -0.25                 0.72                 0.78
promedio_horas_mes            0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
accidente                    -1.25       0.29       0.10            -1.44            -1.05                 0.24                 0.35
ascenso_5years               -1.38       0.25       0.36            -2.08            -0.68                 0.13                 0.51
depa_Investigación           -0.35       0.70       0.12            -0.58            -0.12                 0.56                 0.89
depa_administración          -0.13       0.88       0.15            -0.42             0.16                 0.66                 1.17
depa_contabilidad            -0.11       0.90       0.10            -0.32             0.10                 0.73                 1.10
depa_desarrollo de producto   0.01       1.01       0.10            -0.20             0.21                 0.82                 1.24
depa_informática             -0.06       0.94       0.09            -0.24             0.12                 0.79                 1.13
depa_mantenimiento            0.08       1.08       0.07            -0.05             0.21                 0.95                 1.24
depa_mercadotecnia           -0.01       0.99       0.10            -0.22             0.19                 0.81                 1.21
depa_recursos humanos         0.08       1.09       0.10            -0.12             0.29                 0.89                 1.33
depa_soporte                  0.10       1.11       0.07            -0.04             0.24                 0.96                 1.27
Salario_alto                 -1.54       0.21       0.15            -1.83            -1.25                 0.16                 0.29
Salario_medio                -0.39       0.68       0.05            -0.48            -0.30                 0.62                 0.74

                              cmp to      z      p   -log2(p)
covariate                                                    
satisfaccion                    0.00 -26.89 <0.005     526.58
evaluacion                      0.00  -0.76   0.44       1.17
proyectos                       0.00 -14.75 <0.005     161.15
promedio_horas_mes              0.00   3.88 <0.005      13.25
accidente                       0.00 -12.41 <0.005     115.04
ascenso_5years                  0.00  -3.88 <0.005      13.21
depa_Investigación              0.00  -3.01 <0.005       8.56
depa_administración             0.00  -0.89   0.37       1.42
depa_contabilidad               0.00  -1.05   0.29       1.77
depa_desarrollo de producto     0.00   0.09   0.93       0.11
depa_informática                0.00  -0.65   0.52       0.95
depa_mantenimiento              0.00   1.23   0.22       2.18
depa_mercadotecnia              0.00  -0.11   0.91       0.14
depa_recursos humanos           0.00   0.81   0.42       1.26
depa_soporte                    0.00   1.42   0.16       2.67
Salario_alto                    0.00 -10.36 

# Interpretación 

1. Según aumenta la satisfacción del empleado en la empresa, se reduce en un 22% el riesgo de renuncia.

2. La evaluación no repercute en el riesgo de que el empleado renuncie.

3. Según mas proyectos se le asignen a un empleado, el riesgo de renuncia se reduce en un 25%.

4. Entre más horas promedio trabaje un empleado, su riesgo de renuncia aumenta en un 0.1841%

5. El hecho de que el empleado haya sufrido accidentes en la empresa reduce en un 71% el riesgo de renuncia.

6. El haber recibido un ascenso en los últimos 5 años reduce en un 75% el riesgo de renuncia.

7. Trabajar en el departamento de investigación reduce en un 30% el riesgo de renuncia respecto a trabajar en el departamento de ventas.

8. Trabajar en los siguientes departamentos no tiene repercución en el riesgo de renuncia del empleado.
 depa_administración	
depa_contabilidad	
depa_desarrollo de producto	
depa_informática	
depa_mantenimiento	
depa_mercadotecnia	
depa_recursos humanos	
depa_soporte

9. Tener un salario alto reduce en un 79% el riesgo de renuncia.

10. Tener un salario medio reduce el riesgo de renuncia en un 32%.

In [18]:
cph1.params_

covariate
satisfaccion                  -0.242337
evaluacion                    -0.010687
proyectos                     -0.291606
promedio_horas_mes             0.001839
accidente                     -1.246686
ascenso_5years                -1.380084
depa_Investigación            -0.350547
depa_administración           -0.130262
depa_contabilidad             -0.110158
depa_desarrollo de producto    0.009468
depa_informática              -0.058469
depa_mantenimiento             0.081424
depa_mercadotecnia            -0.011698
depa_recursos humanos          0.083791
depa_soporte                   0.100538
Salario_alto                  -1.538901
Salario_medio                 -0.391289
Name: coef, dtype: float64

In [19]:
cph1.hazard_ratios_

covariate
satisfaccion                   0.784792
evaluacion                     0.989370
proyectos                      0.747063
promedio_horas_mes             1.001841
accidente                      0.287456
ascenso_5years                 0.251557
depa_Investigación             0.704303
depa_administración            0.877865
depa_contabilidad              0.895692
depa_desarrollo de producto    1.009513
depa_informática               0.943208
depa_mantenimiento             1.084830
depa_mercadotecnia             0.988370
depa_recursos humanos          1.087401
depa_soporte                   1.105766
Salario_alto                   0.214617
Salario_medio                  0.676185
Name: exp(coef), dtype: float64

# 5. Con base en la interpretación de los coeficientes, ¿qué recomendación podría hacerle a la empresa para que disminuya la cantidad de renuncias de los empleados?

Recomendaciones.

1. Que la empresa se preocipe más por la satisfacción de sus empleados, ya que esta disminuye en un 22% el riesgo de renuncia entre mas satisfecho está el empleado.

2. Sorprendentemente, el numero de proyectos asignados disminuye en un 25% el riesgo de renuncia, por lo que se recomendaría mantener a los empleados ocupados asignandoles proyectos

3. Aún más sorprendentemente, el haber sufrido accidentes reduce en un 71% el riesgo de renuncia respecto a no haber sufrido accidentes en la empresa. Aquí me abstengo a recomendar promover accidentes en los empleados ya que no sería ético de mi parte.

4. También se recomienda promover los ascensos a los empleados, ya que estos reducen el riesgo de renuncia en un 75%

5. El promedio de horas trabajadas al mes aumenta el riesgo de renuncia en un 0.18%, por lo que se recomendaría no elevar este promedio.

6. Por último, se recomienda promover los salarios medios y altos ya que estos reducen respectivamente el riesgo de renuncia en un 79% y 32% el riesgo de renuncia en comparación a los salarios bajos

# 6. Pruebe si el modelo de Cox cumple el supuesto de riesgos proporcionales, de no hacerlo estime el modelo AFT adecuado

In [20]:
cph1.check_assumptions(renuncias, p_value_threshold=0.05)
## Ho: se viola el supuesto
## Ha: se cumple el supuesto

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
         test_name = proportional_hazard_test

---
                                  test_statistic      p  -log2(p)
Salario_alto                km              0.37   0.54      0.88
                            rank            0.00   0.96      0.06
Salario_medio               km              0.44   0.51      0.98
                            rank            0.75   0.39      1.37
accidente                   km              0.03   0.87      0.21
                            rank            1.14   0.29      1.81
ascenso_5years              km              3.66   0.06      4.17
                            rank            6.71   0.01      6.71
depa_Investigación          km              1.04   0.31      1.70
                            rank            4.36   0.04      4.76
depa_administración         km              3.66   0.06      4.17
                            rank            1.53   0.22      2.21
depa_contabilidad           km              0.52   0.47      1.09
                            rank            0.71   0.40      1.33
depa_desarrollo de producto km              0.07   0.79      0.34
                            rank            1.10   0.29      1.77
depa_informática            km              0.08   0.77      0.37
                            rank            2.59   0.11      3.22
depa_mantenimiento          km              0.31   0.58      0.79
                            rank            0.91   0.34      1.56
depa_mercadotecnia          km              1.24   0.27      1.91
                            rank            1.31   0.25      1.99
depa_recursos humanos       km              0.00   0.99      0.01
                            rank            0.00   1.00      0.00
depa_soporte                km              2.68   0.10      3.30
                            rank            3.25   0.07      3.81
evaluacion                  km            443.05 <0.005    324.32
                            rank          427.15 <0.005    312.82
promedio_horas_mes          km            270.84 <0.005    199.74
                            rank          307.41 <0.005    226.21
proyectos                   km            574.13 <0.005    419.05
                            rank          684.93 <0.005    499.11
satisfaccion                km           1257.74 <0.005    912.74
                            rank          845.24 <0.005    614.90



1. Variable 'satisfaccion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'satisfaccion' might be incorrect. That is, there
may be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'satisfaccion' using pd.cut, and then specify it in
`strata=['satisfaccion', ...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'evaluacion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'evaluacion' might be incorrect. That is, there may
be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to 

[]

Como no se cumple el supuesto, debemos utilizar un Modelo Alternativo, El modelo de riesgo acelerado AFT.

Primero debemos identificar si es Exponencial, Weibull o LogNormal

In [21]:
from lifelines import ExponentialFitter, WeibullFitter, LogNormalFitter

In [22]:
mexpo = ExponentialFitter().fit(renuncias['semestre'], renuncias['renunciar'])
mweibull = WeibullFitter().fit(renuncias['semestre'], renuncias['renunciar'])
mlogn = LogNormalFitter().fit(renuncias['semestre'], renuncias['renunciar'])

In [23]:
#criterio de AIC
mexpo.AIC_, mweibull.AIC_, mlogn.AIC_

(15965.45892201228, 12511.512867661068, 11587.92169210356)

In [24]:
#Se elige el modelo LogNormal ya que tiene menor AIC
from lifelines import LogNormalAFTFitter
LogN_AFT = LogNormalAFTFitter().fit(renuncias, 'semestre', 'renunciar')
LogN_AFT.print_summary()

<lifelines.LogNormalAFTFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
   number of observations = 11991
number of events observed = 1991
           log-likelihood = -4876.57
         time fit was run = 2023-09-25 08:33:11 UTC

---
                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param  covariate                                                                                                                           
mu_    Salario_alto                  0.37       1.45       0.03             0.31             0.43                 1.37                 1.53
       Salario_medio                 0.08       1.09       0.01             0.06             0.11                 1.06                 1.11
       accidente                     0.26       1.30       0.02             0.22             0.30                 1.25                 1.35
       ascenso_5years                0.37       1.44       0.06             0.24             0.49                 1.27                 1.63
       depa_Investigación            0.06       1.06       0.03             0.00             0.11                 1.00                 1.12
       depa_administración           0.11       1.11       0.03             0.04             0.17                 1.04                 1.18
       depa_contabilidad             0.01       1.01       0.03            -0.04             0.06                 0.96                 1.07
       depa_desarrollo de producto   0.01       1.01       0.02            -0.04             0.05                 0.96                 1.06
       depa_informática              0.01       1.01       0.02            -0.03             0.05                 0.97                 1.05
       depa_mantenimiento           -0.02       0.98       0.02            -0.05             0.01                 0.95                 1.01
       depa_mercadotecnia            0.02       1.02       0.03            -0.03             0.07                 0.97                 1.07
       depa_recursos humanos        -0.03       0.97       0.03            -0.08             0.02                 0.92                 1.02
       depa_soporte                 -0.03       0.97       0.02            -0.06             0.01                 0.94                 1.01
       evaluacion                    0.01       1.01       0.00            -0.00             0.01                 1.00                 1.01
       promedio_horas_mes           -0.00       1.00       0.00            -0.00            -0.00                 1.00                 1.00
       proyectos                     0.10       1.10       0.01             0.09             0.11                 1.09                 1.11
       satisfaccion                  0.07       1.07       0.00             0.06             0.07                 1.06                 1.07
       Intercept                     0.91       2.50       0.03             0.86             0.97                 2.36                 2.63
sigma_ Intercept                    -1.11       0.33       0.02            -1.14            -1.08                 0.32                 0.34

                                     cmp to      z      p   -log2(p)
param  covariate                                                    
mu_    Salario_alto                    0.00  12.49 <0.005     116.53
       Salario_medio                   0.00   7.50 <0.005      43.89
       accidente                       0.00  12.98 <0.005     125.59
       ascenso_5years                  0.00   5.80 <0.005      27.19
       depa_Investigación              0.00   2.13   0.03       4.92
       depa_administración             0.00   3.22 <0.005       9.60
       depa_contabilidad               0.00   0.55   0.58       0.79
       depa_desarrollo de producto     0.00   0.22   0.82       0.28
       depa_informática  

In [25]:
LogN_AFT.params_

param   covariate                  
mu_     Salario_alto                   0.369967
        Salario_medio                  0.083861
        accidente                      0.263049
        ascenso_5years                 0.365026
        depa_Investigación             0.057364
        depa_administración            0.105331
        depa_contabilidad              0.014073
        depa_desarrollo de producto    0.005548
        depa_informática               0.010960
        depa_mantenimiento            -0.020282
        depa_mercadotecnia             0.021784
        depa_recursos humanos         -0.028861
        depa_soporte                  -0.025660
        evaluacion                     0.005721
        promedio_horas_mes            -0.000309
        proyectos                      0.096127
        satisfaccion                   0.067044
        Intercept                      0.914441
sigma_  Intercept                     -1.113764
dtype: float64

# 7. Interprete los coeficientes del modelo AFT

1. El salario alto aumenta en un 45% la media del tiempo desde que el empleado es contratado hasta que renuncia respecto al salario bajo.

2. El salario medio aumenta en un 8% la media del tiempo desde que el empleado es contratado hasta que renuncia respecto al salario bajo.

3. El padecer un accidente en la empresa aumenta en un 30% la media del tiempo desde que el empleado es contratado hasta que renuncia respecto a no padecer accidentes en la empresa.

4. El haber tenido un ascenso en los últimos 5 años aumenta en un 44% la media del tiempo desde que el empleado es contratado hasta que renuncia respecto no tener un ascenso.

5. El trabajar en el departamento de investigación aumenta en un 6% la media del tiempo desde que el empleado es contratado hasta que renuncia respecto a trabajar en ventas.

6. El trabajar en el departamento de administración aumenta en un 11% la media del tiempo desde que el empleado es contratado hasta que renuncia respecto a trabajar en ventas.

7. Las siguientes variables no tienen efecto en la media del tiempo de renuncia del empleado: depa_contabilidad
depa_desarrollo de producto	
depa_informática
depa_mantenimiento	
depa_mercadotecnia	
depa_recursos humanos
depa_soporte
evaluacion

8. El aumento del promedio de horas mensuales trabajadas reduce en un muy bajo porcentaje la media del tiempo desde que el empleado es contratado hasta que renuncia.

9. El aumento de numero de proyectos asigandos a los empleados aumenta en un 10% la media del tiempo desde que el empleado es contratado hasta que renuncia.

10. El aumento de la satisfacción del empleado aumenta en un 7% la media del tiempo desde que el empleado es contratado hasta que renuncia.
